# Projeto Webscraping: Cálculo de Rentabilidade Diária e Anual de Fundos de Ações

**Autor:** Eduardo B. Scheffer

**Data:** 31/08/2020

**Objetivo:** Obter rentabilidade diária e YTD de uma lista de fundos de ações:

* Constellation Ações FIC FIA
* Sqadra Long-Only FIC FIA
* Bogari Value FIC FIA
* Núcleo FIC FIA
* Dynamo Cougar FIA

**Observação:** Abrir o arquivo "Projeto Brasil Capital.xlsx" antes de e após rodar este programa, para verificar sua funcionalidade

---
## Método I:
    Extração dos dados das cotas diretamente de fontes públicas

### Importar bibliotecas:

In [2]:
import pandas as pd
from datetime import date
from pandas import ExcelWriter
# !pip install openpyxl --> "descomente" essa linha caso não tenha o openpyxl instalado
from openpyxl import load_workbook

### Declarar CNPJ dos fundos:

In [3]:
fundos = {'Nome Fundo': ['Constellation Ações FIC FIA', 'Sqadra Long-Only FIC FIA', 'Bogari Value FIC FIA', 'Núcleo FIC FIA', 'Dynamo Cougar FIA'],
          'CNPJ': ['16.948.298/0001-04', '09.412.822/0001-54', '08.323.402/0001-39', '14.068.366/0001-07', '73.232.530/0001-39']         
         }
fundos = pd.DataFrame(fundos, columns = ['Nome Fundo', 'CNPJ'])
fundos['Rentabilidade diária'] = ""
fundos['Rentabilidade YTD'] = ""

### Obter dados através da CVM - Dados Abertos

In [4]:
def consulta_cvm(x):
    url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}.csv'.format(x)
    df = pd.read_csv(url, sep=";", usecols = ["CNPJ_FUNDO", "DT_COMPTC", "VL_QUOTA"])
    return df

### Obter cotas de fundos da CVM referentes do ano atual (YTD)

In [ ]:
year = date.today().year # Ano atual
month = date.today().month # Mês atual
day = date.today().day # Dia atual

if day ==1: month = month - 1

for m in range (1,month+1):
    
    yyyy = str(year)
    if m < 10: mm = '0'+str(m)
    else: mm = str(m)
    
    data = yyyy+mm # Ano e mes para substituir no url
    
    cvm_m = consulta_cvm(data)
    if m == 1:
        firstD = cvm_m['DT_COMPTC'].iloc[0] # Verifica qual é o primeiro dia do ano
        lastD = cvm_m['DT_COMPTC'].iloc[len(cvm_m)-1] # Verifica qual é o primeiro dia do ano
        cvm = cvm_m
    else:
        cvm = pd.concat([cvm,cvm_m])

### Obter as cotas dos fundos selecionados

In [ ]:
quotas_fundos = {} #array de dataframes
for x in range(0, len(fundos)):
    cnpj = fundos['CNPJ'].iloc[x]
    quotas_fundos[x] = pd.DataFrame(cvm.loc[cvm['CNPJ_FUNDO'] == cnpj]) # Copia nova df para cada fundo com as informações da CVM relativos ao seu CNPJ
    quotas_fundos[x].drop('CNPJ_FUNDO', axis=1, inplace=True) # Deleta coluna do CNPJ para "limpar" a df (já que todas as linhas serão iguais)
    quotas_fundos[x].reset_index(drop=True,inplace=True) # Renumera os indices, para começar do 0

    

### Calcular as variações percentuais diárias da cota de cada fundo

In [ ]:
for x in range(0, len(quotas_fundos)):
    quotas_fundos[x].insert(len(quotas_fundos[x].columns),'VAR_DIAR','') # Insere nova coluna para a rentabilidade diária
    quotas_fundos[x].insert(len(quotas_fundos[x].columns),'VAR_CUMU','') # Insere nova coluna para a rentabilidade cumulativa
    i=0
    while i+1 < len(quotas_fundos[x]):
        if i == 0:
            quotas_fundos[x].at[i,'VAR_DIAR'] = 0 # Normaliza a primeira data com rentabilidade diária 0
            quotas_fundos[x].at[i,'VAR_CUMU'] = 0 # Normaliza a primeira data com rentabilidade acumulada 0
        i = i+1;
        rd1 = float(str(quotas_fundos[x].at[i-1,'VL_QUOTA'])) # Valor da cota do dia anterios
        rd2 = float(str(quotas_fundos[x].at[i,'VL_QUOTA'])) # Valor da cota do dia
        vd = rd2 / rd1 - 1
        quotas_fundos[x].at[i,'VAR_DIAR'] = vd # Calcula a variação do valor da cota (rentabilidade)
        quotas_fundos[x].at[i,'VAR_CUMU'] = (1 + vd) * (1 + quotas_fundos[x].at[i-1,'VAR_CUMU']) -1 # Calcula a variação do valor da cota (rentabilidade)
        

### Calcular a rentabilidade diária mais recente e a rentabilidade do ano (YTD) para os fundos selecionados

In [ ]:
for x in range(0, len(fundos)):
    l = len(quotas_fundos[x])
    fundos['Rentabilidade diária'].iloc[x] =  quotas_fundos[x].at[l-1,'VAR_DIAR']
    fundos['Rentabilidade YTD'].iloc[x] =  quotas_fundos[x].at[l-1,'VAR_CUMU']

### Print do Resultado:

In [81]:
fundos

NameError: name 'fundos' is not defined

### Exportar para o Excel em nova aba ("Input")

In [ ]:
wb = load_workbook("Projeto Brasil Capital.xlsx") #Abre a planilha a ser preenchida

writer = pd.ExcelWriter("Projeto Brasil Capital.xlsx")
writer.book = wb
writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # Possibilita escrever em aba já existente em caso de atualização dos dados

fundos.to_excel(writer, sheet_name='Input', index = False) # Copia informações do fundo na aba "Input"
writer.save()

### Exportar para o Excel em na aba "Projeto_2020"

In [ ]:
ws = wb['Projeto_2020']
for x in range(0, len(fundos)):
    ws.cell(row = 12+x, column=4).value = fundos['Rentabilidade diária'].iloc[x]
    ws.cell(row = 12+x, column=5).value = fundos['Rentabilidade YTD'].iloc[x]
writer.save()

---
## Método II:
    Extração dos dados das cotas diretamente do site das gestoras
    
   #### Obs.:

    - Constellation não disponibiliza informações das cotas dos fundos ou rentabilidade em seu website.
    
    - Dynamo não possui tabelas em seu website. As ferramentas que eu tinha à disposição não me permitiram extrair essas informações dentro de um tempo razoável 
    
    - Sqadra, Bogari e Nucleo não disponibilizam apenas rentabilidade diária
    
   #### Portanto, serão obtidas as rentabilidades acumuladas do Sqadra Long-Only FIC FIA, Bogari Value FIC FIA e Núcleo FIC FIAe a rentabilidade do dia do Dynamo Cougar FIA através deste método

### Importar bibliotecas:

In [1]:
import pandas as pd
from pandas import ExcelWriter
from datetime import date
from openpyxl import load_workbook

### Definição dos url das gestoras.



In [2]:
asset_url = ["https://constellation.com.br/pra-voce/acoes/", "http://www.squadrainvestimentos.com/fundo-longonly.aspx", "http://bogaricapital.com.br/fundos/fundo-modelo/",
               "https://www.nucleocapital.com.br/nucleo-acoes-fic-fia.php", "https://www.dynamo.com.br/en"]

In [3]:
fundos2 = {'Nome Fundo': ['Constellation Ações FIC FIA', 'Sqadra Long-Only FIC FIA', 'Bogari Value FIC FIA', 'Núcleo FIC FIA', 'Dynamo Cougar FIA']}
fundos2 = pd.DataFrame(fundos2, columns = ['Nome Fundo'])
fundos2['Rentabilidade diária'] = ""
fundos2['Rentabilidade YTD'] = ""

In [78]:
for x in range(0, len(fundos2)):
    
    # Cada asset é um caso diferente, pois cada website é diferente em termos de tabelas 
    # Sqadra
    if x == 1:  
        tables = pd.read_html(asset_url[x]) # cria um data frame com as tabelas presentes no site 
        df = tables[0]  # Utiliza-se a primeira tabela
        df =  df.iloc[1:] # Limpa a primeira linha
        df = df[df[0].notna()] # Deleta as linhas vazias
        df.reset_index(drop=True,inplace=True) # Renumera os indices, para começar do 0
        for i in range(len(df)) :
            if(df.at[i,0]) == '2020': 
                fundos2['Rentabilidade YTD'].iloc[x] = float(str(df.at[i,13])) / 10000 # Obtém-se a rentabilidade acumulada de 2020
                break
                
    # Bogari
    if x == 2: 
        tables = pd.read_html(asset_url[x])
        df = tables[0]  # tiliza-se a primeira tabela
        for i in range(len(df)) :
            if(df[df.index == i].iloc[i,0]) == '2020': 
                rentab = df[df.index == i].iloc[i,13]
                rentab = rentab.replace(",",".") ## agora, transformando string para float e então, para porcentagem
                rentab = rentab.replace("%","")
                fundos2['Rentabilidade YTD'].iloc[x] = float(rentab) / 100
                break      
           
    # Núcleo
    if x == 3: 
        tables = pd.read_html(asset_url[x]) # cria um data frame com as tabelas presentes no site 
        df = tables[0]  # Utiliza-se a primeira tabela
        for i in range(len(df)) :
            if(df[df.index == i].iloc[0,0]) == 2020: 
                rentab = df[df.index == i].iloc[0,13]
                rentab = rentab.replace(",",".") ## agora, transformando string para float e então, para porcentagem
                rentab = rentab.replace("%","")
                fundos2['Rentabilidade YTD'].iloc[3] = float(rentab) / 100
                break

In [82]:
### Print dos resultados:

In [79]:
fundos2

,Nome Fundo,Rentabilidade diária,Rentabilidade YTD
0,Constellation Ações FIC FIA,,
1,Sqadra Long-Only FIC FIA,,-0.0244
2,Bogari Value FIC FIA,,0.004
3,Núcleo FIC FIA,,0.0651
4,Dynamo Cougar FIA,,


### Exportar para o Excel em nova aba ("Input")

In [84]:
wb = load_workbook("Projeto Brasil Capital.xlsx") #Abre a planilha a ser preenchida

writer = pd.ExcelWriter("Projeto Brasil Capital.xlsx")
writer.book = wb
writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # Possibilita escrever em aba já existente em caso de atualização dos dados

fundos2.to_excel(writer, sheet_name='Input', index = False) # Copia informações do fundo na aba "Input"
writer.save()

### Exportar para o Excel em na aba "Projeto_2020"

In [88]:
ws = wb['Projeto_2020']
for x in range(0, len(fundos2)):
    ws.cell(row = 12+x, column=4).value = fundos2['Rentabilidade diária'].iloc[x]
    ws.cell(row = 12+x, column=5).value = fundos2['Rentabilidade YTD'].iloc[x]
writer.save()